# Optical character recognition (OCR) 
Project 4 Group 5


## Error Detection

First of all, we need to detect errors, or incorrectly processed words. Here we extract features according to the paper and use SVM for garbage detection.

In [73]:
import os
import glob
import re
from collections import Counter
import pandas as pd
import collections
import numpy as np
from sklearn.svm import SVC
import itertools
#import itertools
#nltk.download('punkt')

In [74]:
##### read ground_truth
ground_dir = glob.glob(os.path.join(os.getcwd(),'../data/ground_truth','*.txt'))

In [85]:
# make sure corresponding files have the same number of lines
tess_dir = glob.glob(os.path.join(os.getcwd(),'../data/tesseract','*.txt'))
file_name_gd = []
file_name_td = []
for gd, td in zip(ground_dir, tess_dir):
        with open(gd, encoding="utf8") as ground_file:    #, encoding="utf8"
            with open(td, encoding="utf8") as tess_file:                
                ground_r = list(ground_file.readlines()) 
                tess_r = list(tess_file.readlines())
                if len(tess_r) == len(ground_r):
                    file_name_td.append(td)
                    file_name_gd.append(gd)

In [86]:
# make sure all the lines have the same size.
# recreate ground_tokens and tess_tokens
ground_tokens=[]
tess_tokens=[]
for gd, td in zip(file_name_gd, file_name_td):
        with open(gd) as file1:
            with open(td) as file2:
                for line1,line2 in zip(file1,file2):
                    if len(line1)==len(line2):
                        #if length is different, skip that line
                        for word1,word2 in zip(line1.split(),line2.split()):
                            ground_tokens.append(word1)
                            tess_tokens.append(word2)
#ground_tokens and tess_tokens are a list of words

In [77]:
print(tess_tokens[:30])
print(ground_tokens[:30])

['communlcatlons', 'network.', 'Member', 'companles', 'are', 'strongly', 'encouraged', 'to', 'provlde', 'thls', 'needed', 'support.', 'The', 'state', 'advocacy', 'program"', '1nclud1ng', 'the', 'new', 'CMA/LINC', 'computer', 'network.', 'will', 'be', 'heavlly', '1nvolved', '1n', '1995', '1n', 'the']
['communications', 'network.', 'Member', 'companies', 'are', 'strongly', 'encouraged', 'to', 'provide', 'this', 'needed', 'support.', 'The', 'state', 'advocacy', 'program*', 'including', 'the', 'new', 'CMA/LINC', 'computer', 'network,', 'will', 'be', 'heavily', 'involved', 'in', '1986', 'in', 'the']


In [78]:
print(len(tess_tokens))
print(len(ground_tokens))

152143
152143


In [79]:
# for every word in tess_tokens, y indicates the correctness of the word
y = []
for gt, tt in zip(ground_tokens, tess_tokens):
        if gt == tt:
            y.append(0)   # 0 indicates correct
        else:
            y.append(1)   # 1 indicates error

In [80]:
X = pd.DataFrame(tess_tokens)   
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
ground_train = [ground_tokens[i] for i in X_train.index.tolist()]
ground_test = [ground_tokens[i] for i in X_test.index.tolist()]
X_train = X_train[0].tolist()
X_test = X_test[0].tolist()

In [81]:
print(X_train[:30])
print(ground_train[:30])
print(y_train[:30])

['hundreds.', 'chemlcal', 'total', 'an', 'recalled', 'chemlcal', 'young', '1992793.', 'to', 'technlcal', 'have', 'products.', '1:', 'been', '(except', 'be', 'change', 'testlng', 'prlorlty', 'h.d', 'from', 'them', 'Vinyl', 'out', 'the', 'a.', 'organlzatlon', 'Forum', 'carry', 'antltrust']
['hundreds,', 'chemical', 'total', 'an', 'recalled', 'chemical', 'young', '1982-83.', 'to', 'technical', 'have', 'products.', 'it', 'been', '(except', 'be', 'change', 'testing', 'priority', 'had', 'from', 'them', 'vinyl', 'out', 'the', 'H.', 'organization', 'Forum', 'carry', 'antitrust']
[1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1]


In [82]:
# define character type
vowels = list('aeiou')
consonants = list("bcdfghjklmnpqrstvwxyz")
digits = list('0123456789')

# Rule 9: If the word contains the # of upper case characters > # of lower case character => garbadge
# count # of upper case characters 
def count_up(word):
    upper = [c for c in word if c.isupper()]
    return len(upper)
# count # of lower case characters
def count_low(word):
    lower = [c for c in word if c.islower()]
    return len(lower)
# Rule 8: If the string contains at least 3 consecutive of same symbol => garbadge 
# count # consecutive occurrences of the same symbol
def count_cons_occur_symbol(word): 
    count = 0 
    tmp = 0 
    curr = ''
    for c in word:
        if curr==c:
            tmp +=1
        else:
            count = max(tmp,count)
            tmp = 1 
            curr = c
    count = max(tmp, count)            
    return count 
   
# Rule 11: if the word contains 4 consecutive vowels or 5 consecutive consonants => garbage
#consecutive occurrences of the same consonants
def count_cons_occur_consonants(word):  
    max_count=0
    count = 0
    curr = ''
    for c in word:
        if c in consonants:
            if curr == c:
                count += 1
            else:
                max_count = max(max_count, count)
                curr = c
                count = 1
    max_count = max(max_count, count)
    return(max_count)

#extract a subset of word
def trim_word(word, start=1, end=1):
    return word[start:len(word) - end]

            
def safe_div(x,y):
    if y == 0:
        return 0
    return x / y


def get_bigram_freq(word, bi_dict):
    word = word.lower()
    bf = []
    for i in range(len(word)-1):
        key = word[i:i+2]
        bf.append(bi_dict[key])
    return(bf)

In [83]:
### find LB in ground_truth, prepared for bigram features
# ground_tokens of all words in lowercase
lower_ground_tokens = []
for tk in ground_tokens:
    tkl_g = tk.lower()
    lower_ground_tokens.append(tkl_g)

# A dict of all bigram frequencies
bigram_dict = collections.defaultdict(int)
for tk_g in lower_ground_tokens:
    for i in range(len(tk_g)-1):
        key = tk_g[i:i+2]
        bigram_dict[key] += 1
#bigram_dict is a dictionary of bigram frequence

In [84]:
bigram_dict

defaultdict(int,
            {'co': 7987,
             'om': 4529,
             'mm': 2163,
             'mu': 645,
             'un': 2570,
             'ni': 2774,
             'ic': 5599,
             'ca': 4529,
             'at': 12092,
             'ti': 12863,
             'io': 8332,
             'on': 15058,
             'ns': 4286,
             'ne': 3808,
             'et': 2546,
             'tw': 328,
             'wo': 1168,
             'or': 8733,
             'rk': 693,
             'k.': 41,
             'me': 5973,
             'em': 4018,
             'mb': 941,
             'be': 3693,
             'er': 10941,
             'mp': 2223,
             'pa': 3439,
             'an': 12007,
             'ie': 2439,
             'es': 9802,
             'ar': 6346,
             're': 13219,
             'st': 7403,
             'tr': 3531,
             'ro': 6615,
             'ng': 5832,
             'gl': 97,
             'ly': 1988,
             'en': 10468,
         

In [41]:
def feature_matrix(X):
    #input: a list of words
    #Output: a dataframe included features, every feature is a list
    
    ##### Feature extracting
    # feature 1 
    length = []

    # feature 2
    vowels_count = []
    consonants_count = []
    quotients_v_l = []
    quotients_c_l = []
    quotients_v_c = []

    # feature 3
    nonalpha = []

    # feature 4
    digits = []
    quotients_d_l = []

    # feature 5
    lowers = []
    uppers = []
    quotients_low_l = []
    quotients_up_l = []
    
    #feature 6
    consecutive_occur = []
    quotients_cons_l = []
    
    #feature 7
    quotients_la = [] # total number of vowels, consonants and digits
    
    #feature 8
    consonants_secut = []
    
    #feature 9
    fix_nonalpha = []

    # feature bigram
    bigr = []

    #most frequent symbol
    most_freq = []
    
    #non-alphabetical symbols
    l2 = []
    
    for tk in X:
        ### feature 1
        l = len(tk)
        length.append(l)

        ### feature 2
        tkl = tk.lower()
        v_count = 0
        for v in tkl:
            if v in vowels:
                v_count +=1
        vowels_count.append(v_count)

        c_count = 0
        for c in tkl:
            if c in consonants:
                c_count +=1
        consonants_count.append(c_count)

        quotients_v_l.append(v_count/l)
        quotients_c_l.append(c_count/l)
        quot_v_c = safe_div(v_count,c_count)
        quotients_v_c.append(quot_v_c)

        ### feature 4
        d_count = len([d for d in tk if d in digits])
        digits.append(d_count)
        quotients_d_l.append(d_count/l)

        ### feature 3
        s_count = len([s for s in tk if s not in vowels or consonants or digits])
        nonalpha.append(s_count)

        ### feature 5
        low_count = count_low(tk)
        lowers.append(low_count)
        up_count = count_up(tk)
        uppers.append(up_count)
        quotients_low_l.append(low_count/l)
        quotients_up_l.append(up_count/l)

        ### feature 6
        cons_occur_count = int(count_cons_occur_symbol(tk))
        consecutive_occur.append(cons_occur_count)
        if cons_occur_count >= 3:
            quotients_cons_l.append(cons_occur_count/l)
        else:
            quotients_cons_l.append(0)

        ### feature 7
        la = v_count + c_count + d_count
        if s_count > la:
            quotients_la.append(1)
        else:
            quotients_la.append(0)

        ### feature 8
        consonance_cons_cccur_count = int(count_cons_occur_consonants(tk))
        if consonance_cons_cccur_count >= 6:
            consonants_secut.append(1)
        else:
            consonants_secut.append(0)

        ### feature 9
        tk_removed = trim_word(tk)
        k_count = len([k for k in tk_removed if k not in vowels or consonants or digits])
        if k_count >=3:
            fix_nonalpha.append(1)
        else:
            fix_nonalpha.append(0)  

        ### feature 10 bigram
        bf = get_bigram_freq(tk, bigram_dict)

        lower_tess_tokens = []
        tkl_t = tk.lower()
        lower_tess_tokens.append(tkl_t)

        n = len(set(lower_tess_tokens))
        big = (sum(bf)/10000)/n 
        bigr.append(big)

        ### feature 11 most frequent symbol
        i_count = Counter(tk).most_common(1)[0][1]
        if i_count >=3:
            most_freq.append(1)
        else:
            most_freq.append(0)

        ### feature 12 Non-alphabetical symbols: nonalpha/total symbols
        l1 = len([v for v in tk.lower() if v in vowels] + [c for c in tk.lower() if c in consonants])
        l2_count = l - l1
        quot_l2 = safe_div(l2_count,l1)
        l2.append(quot_l2)  
        
    ##### construct a feature dataframe for SVM
    df1 = pd.DataFrame({'length': length,
                        'vowels': vowels_count,
                        'consonants': consonants_count,
                        'quot v/l': quotients_v_l,
                        'quot c/l': quotients_c_l,
                        'quot v/c': quotients_v_c,
                        'nonalpha': nonalpha,
                        'digits': digits,
                        'quot d/l': quotients_d_l,
                        'lowers': lowers,
                        'uppers': uppers,
                        'quot low/l': quotients_low_l,
                        'quot up/l': quotients_up_l,
                        'cons occur': consecutive_occur,
                        'quot cons/l': quotients_cons_l,
                        'quot la': quotients_la,
                        'consonants_secut': consonants_secut,
                        'fix_nonalpha': fix_nonalpha,
                        'bigr': bigr,
                        'most_freq' : most_freq,
                        'l2': l2})
    return df1

In [42]:
#it takes about 10 mins
X_feature_train = feature_matrix(X_train)
X_feature_test = feature_matrix(X_test)

In [43]:
X_feature_train.to_csv('../output/X_feature_train')
X_feature_test.to_csv('../output/X_feature_test')

In [44]:
#it takes about 20 mins
svclassifier = SVC(kernel='rbf')  
svclassifier.fit(X_feature_train, y_train)  

/Users/shengweihuang/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [45]:
#save model
import pickle
filename = '../output/SVM_model.sav'
pickle.dump(svclassifier, open(filename, 'wb'))

In [46]:
#this chunk is for load model
import pickle
filename = '../output/SVM_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [47]:
loaded_model

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [48]:
#to predict
y_pred = svclassifier.predict(X_feature_test)  
# y_pred = svm_model.predict(X_feature_test)  

In [49]:
df_output = pd.DataFrame({'tokens_tesseract':X_test,
                          'Predict_by_SVM': y_pred})
print(df_output[:100])

   tokens_tesseract  Predict_by_SVM
0        initiative               0
1           Charles               0
2                 s               1
3                '5               1
4              fall               0
5                en               0
6             3111:               1
7            repeal               0
8            rlghts               1
9                19               1
10           Monday               0
11              our               0
12          leaders               0
13              EMC               0
14             Such               0
15      Interagency               1
16               be               0
17         Vlolates               0
18      preparatlon               1
19               1n               1
20           showed               0
21             fees               0
22               on               0
23           method               0
24             Last               0
25               on               0
26        emlsslons         

In [50]:
##### evaluation
#confustion Matrix
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  

[[16206  2142]
 [ 3484  8597]]
              precision    recall  f1-score   support

           0       0.82      0.88      0.85     18348
           1       0.80      0.71      0.75     12081

   micro avg       0.82      0.82      0.82     30429
   macro avg       0.81      0.80      0.80     30429
weighted avg       0.81      0.82      0.81     30429



### MLP

In [51]:
#MLP performs bad
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_feature_train, y_train) 

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [52]:
import pickle
filename = '../output/MLP_model.sav'
pickle.dump(clf, open(filename, 'wb'))

y_pred = clf.predict(X_feature_test)
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred)) 

[[18348     0]
 [12081     0]]
              precision    recall  f1-score   support

           0       0.60      1.00      0.75     18348
           1       0.00      0.00      0.00     12081

   micro avg       0.60      0.60      0.60     30429
   macro avg       0.30      0.50      0.38     30429
weighted avg       0.36      0.60      0.45     30429



/Users/shengweihuang/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


 ### KNN

In [53]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_feature_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [54]:
import pickle
filename = '../output/KNN_model.sav'
pickle.dump(neigh, open(filename, 'wb'))

y_pred = neigh.predict(X_feature_test)
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred)) 

[[17416   932]
 [ 2156  9925]]
              precision    recall  f1-score   support

           0       0.89      0.95      0.92     18348
           1       0.91      0.82      0.87     12081

   micro avg       0.90      0.90      0.90     30429
   macro avg       0.90      0.89      0.89     30429
weighted avg       0.90      0.90      0.90     30429



### random forest

In [55]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=10,min_samples_split=2, random_state=0)
RF = RF.fit(X_feature_train, y_train)

In [56]:
import pickle
filename = '../output/RF_model.sav'
pickle.dump(neigh, open(filename, 'wb'))

y_pred = neigh.predict(X_feature_test)
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred)) 

[[17416   932]
 [ 2156  9925]]
              precision    recall  f1-score   support

           0       0.89      0.95      0.92     18348
           1       0.91      0.82      0.87     12081

   micro avg       0.90      0.90      0.90     30429
   macro avg       0.90      0.89      0.89     30429
weighted avg       0.90      0.90      0.90     30429



## Error correction(I didn't run this part--Xishi Chen)

Given the detected word error, in order to find the best correction, we need to generating the candidate corrections: a dictionary or a database of legal n-grams to locate one or more potential correction terms. Then we need invoke some lexical-similarity measure between the misspelled string and the candidates or a probabilistic estimate of the likelihood of the correction to rank order the candidates. Here, we implement the positional binary digram method in the first reference paper. (https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=1672564}{positional binary digram)

In [57]:
import string

In [58]:
def keep_alphabet(tokens):
  # only retain alphabet
  out = []
  for l in tokens:
      l = l.lower()
      if l in set('abcdefghijklmnopqrstuvwxyz '):
          out.append(l)
  return ''.join(out)

In [59]:
# Replace a postion of a string
def replace_str_index(text,index=0,replacement=''):
    return '%s%s%s'%(text[:index],replacement,text[index+1:])

In [60]:
# Define the main correction function
def correction(word, digrams):
  detect = 0
  beta = []
  # Find the error positions
  for each in digrams:
    matrix = digrams[each]
    if matrix[string.ascii_lowercase.index(word[each[0]])][string.ascii_lowercase.index(word[each[1]])] == 0:
      detect += 1
      beta_each = set(each)
      if detect == 1:
        beta = beta_each
      else:
        beta = beta.intersection(beta_each)
  # print(beta)
  # Consider when there is one or two elements in beta
  if len(beta) in [1,2]:
    choices = 0
    v_dict = {}
    for i in beta:
      v_list = []
      position = i
      for j in range(len(word)):
        alpha_j = string.ascii_lowercase.index(word[j])
        if j < position:
          vector_j = digrams[(j, position)][alpha_j]
          v_list.append(vector_j)
        elif j > position:
          vector_j = [item[alpha_j] for item in digrams[(position, j)]]
          v_list.append(vector_j)
      v = v_list[0]
      for each in v_list:
        v = [a and b for a, b in zip(v, each)]
      if sum(v) == 1:
        choices += 1
        v_dict[i] = v
    if choices == 1:
      for key in v_dict:
        position_chosen = key
      word = replace_str_index(word, position_chosen, string.ascii_lowercase[v_dict[position_chosen].index(1)])
  return word

In [61]:
for i in range(len(ground_train)):
  ground_train[i] = keep_alphabet(ground_train[i])

In [62]:
# Create dictionary of digrams
token_by_len = collections.defaultdict(list)
digrams_by_len = collections.defaultdict(dict)
for w in ground_train:
  token_by_len[len(w)].append(w)
  
#print('Number of words of diffenrent length:')
#for key, value in token_by_len.items() :
#    print (key, len(value))

for length in token_by_len:
  for i, j in itertools.combinations(range(length), 2):
    key = (i, j)
    matrix = [[0] * 26 for _ in range(26)]
    for words in token_by_len[length]:
      matrix[string.ascii_lowercase.index(words[i])][string.ascii_lowercase.index(words[j])] = 1
    digrams_by_len[length][key] = matrix

In [63]:
# Number of 1 in digrams with given length
ae = 0
for each in digrams_by_len[10][(0,1)]:
  ae += sum(each)
print(ae)

144


In [64]:
# Clean the words
corrected_test = X_test.copy()
for i in range(len(corrected_test)):
  corrected_test[i] = keep_alphabet(corrected_test[i])
for i in range(len(ground_test)):
  ground_test[i] = keep_alphabet(ground_test[i])

In [65]:
# Make correction
for i in range(len(y_pred)):
  if y_pred[i] == 1:
    word_length = len(corrected_test[i])
    if word_length > 1:
      digrams_i = digrams_by_len[word_length]
      corrected_test[i] = correction(corrected_test[i], digrams_i)

In [66]:
y_corrected = []
for gt, ct in zip(ground_test, corrected_test):
        if gt == ct:
            y_corrected.append(0)   # 0 indicates correct
        else:
            y_corrected.append(1)   # 1 indicates error

In [67]:
# Compare the results
print(corrected_test[:30])
print(ground_test[:30])
print(X_test[:30])

['initiative', 'charles', 's', '', 'fall', 'en', '', 'repeal', 'rlghts', '', 'monday', 'our', 'leaders', 'emc', 'such', 'interagency', 'be', 'vlolates', 'preparatlon', 'n', 'showed', 'fees', 'on', 'method', 'last', 'on', 'emlsslons', 'from', 'hlth', 'attached']
['was', 'charles', 'foodrelated', 'board', 'fall', 'enacted', 'bills', 'repeal', 'rights', '', 'monday', 'our', 'leaders', 'emc', 'such', 'interagency', 'be', 'violates', 'preparation', 'in', 'showed', 'fees', 'on', 'method', 'last', 'on', 'emissions', 'a', 'with', 'attached']
['initiative', 'Charles', 's', "'5", 'fall', 'en', '3111:', 'repeal', 'rlghts', '19', 'Monday', 'our', 'leaders', 'EMC', 'Such', 'Interagency', 'be', 'Vlolates', 'preparatlon', '1n', 'showed', 'fees', 'on', 'method', 'Last', 'on', 'emlsslons', 'from', 'Hlth', 'attached']


In [68]:
print(sum(y_test))
print(sum(y_corrected))

12081
10357


## Performance measure

In [69]:
# Create corresponding lists of characters
corrected_char = []
for each in corrected_test:
  corrected_char += each
corrected_char

ground_char = []
for each in ground_test:
  ground_char += each

tess_char = [] 
tess_test = X_test.copy()
for i in range(len(tess_test)):
  tess_test[i] = keep_alphabet(tess_test[i])
for each in tess_test:
  tess_char += each

In [70]:
import numpy as np

In [71]:
# Count the number of characters
ground_count = []
corrected_count = []
tess_count = []
for each in string.ascii_lowercase:
  ground_count.append(ground_char.count(each))
  corrected_count.append(corrected_char.count(each))
  tess_count.append(tess_char.count(each))
ground_corrected_min = np.minimum(ground_count, corrected_count)
ground_tess_min = np.minimum(ground_count, tess_count)

In [72]:
# Make the evaluation table
word_recall_tess = 1 - sum(y_test)/len(ground_test)
word_recall_corrected = 1 - sum(y_corrected)/len(ground_test)
word_precision_tess = 1 - sum(y_test)/len(X_test)
word_precision_corrected = 1 - sum(y_corrected)/len(X_test)
char_recall_tess = sum(ground_tess_min)/sum(ground_count)
char_precision_tess = sum(ground_tess_min)/sum(tess_count)
char_recall_corrected = sum(ground_corrected_min)/sum(ground_count)
char_precision_corrected = sum(ground_corrected_min)/sum(corrected_count)

d = {'Tesseract': [word_recall_tess, word_precision_tess, char_recall_tess, char_precision_tess], 
     'Tesseract_with_postprocessing': [word_recall_corrected, word_precision_corrected 
                                       ,char_recall_corrected, char_precision_corrected]}
OCR_performance_table = pd.DataFrame(data=d)
OCR_performance_table.rename(index={0: 'word_wise_recall', 1:'word_wise_precision', 
                                    2:'character_wise_recall', 3:'character_wise_precision'}, inplace=True)
print(OCR_performance_table)

                          Tesseract  Tesseract_with_postprocessing
word_wise_recall           0.602977                       0.659634
word_wise_precision        0.602977                       0.659634
character_wise_recall      0.912228                       0.917197
character_wise_precision   0.946315                       0.951469


The word-wise recall after postprocessing is 0.6595, significantly better than original Tesseract text. Word-wise precision is the same as recall because of our preprocessing. The character-wise recall and precision are slightly improved to 0.9176 and 0.951.